# テーブル概要
accident:  
event:  
gv(general vehicle):車両一般  
ve(Exterior Vehicle):車両外部  
vi(Interior Vehicle):車両内部  
oa(OCCUPANT ASSESSMENT):乗員の調査  
oi(OCCUPANT INJURY):乗員の傷害(mergeに使用できるkeyの値が同一でも傷害箇所によってレコードが増加)  

# 最終的な作成データ
- Crash year 2010–2015
- Vehicle model year 2001–2015
- Light vehicles (passenger cars, pick-ups and mini-vans) 
- Non-ejected occupants
- Occupant age 15 or higher
- Occupants with known injury status or fatality

# ライブラリのインポート Pandasの表示設定
同一cellに複数テーブルを表示  
全カラムを表示  
最大表示行数:500  
1つのカラムの最大表示文字数:200  
floatの有効桁数:4  
色付き文字の出力:print(pycolor.RED + '文字列' + pycolor.END)  

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import sys
from IPython import embed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:.4g}'.format
class pycolor:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    PURPLE = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    END = '\033[0m'
    BOLD = '\038[1m'
    UNDERLINE = '\033[4m'
    INVISIBLE = '\033[08m'
    REVERCE = '\033[07m'

# NASS CDSデータの読み込み

In [2]:
path = os.path.dirname(os.path.abspath('__file__'))
file_name = ['accident', 'event', 'gv', 'oa', 'oi', 've', 'vi']#, 'makmod'
cds_key = []
cds = {}
for year in range(2010, 2016):
    for file in file_name:
        df = pd.read_sas(os.path.join(path, str(year), 'FormattedData', '{}.sas7bdat'.format(file)))
        cds_key.append('{}_{}'.format(file, year - 2000))
        cds['{}_{}'.format(file, year - 2000)] = df

In [3]:
path = os.path.dirname(os.path.abspath('__file__'))
expand_name = ['airbag', 'bagseat', 'childseat', 'seatloc', 'tire', 'tiredmg']
exp_key = []
exp = {}
for year in range(2011, 2016):
    for expand in expand_name:
        try:
            df = pd.read_sas(os.path.join(path, str(year), 'ExpandedSAS', '{}.sas7bdat'.format(expand)))
        except FileNotFoundError:
            df = pd.read_sas(os.path.join(path, str(year),  'ExpandedSAS', 'UNFORMATTED', '{}.sas7bdat'.format(expand)))
        exp_key.append('{}_{}'.format(expand, year - 2000))
        exp['{}_{}'.format(expand, year - 2000)] = df

# 特定のカラムが存在するか確認
引数：DFを格納した辞書, 有無を確かめたいカラムのリスト, オプション:調べたいDFのkey(デフォルトは全てのDF)

In [4]:
def exist_check(df_dic, check_columns, df_keys = None):
    if df_keys is None:
        df_keys = df_dic.keys()
    return_df = pd.DataFrame()
    for key in df_keys:
        df_columns = df_dic[key].columns.values
        check_dic = {}
        for check_column in check_columns:
            check_flag = 0
            for df_column in df_columns:
                if df_column == check_column:
                    check_flag = 1
            check_dic[check_column] = check_flag
        check_df = pd.io.json.json_normalize(check_dic)
        check_df = check_df.rename(index = {0: key})
        return_df = pd.concat([return_df, check_df])
    return_df.head(len(return_df))
    return return_df

# 同名のDFで，年毎に異なるカラムを持つか確認
年によって異なるカラムを持つファイル名とそのカラム名を出力する

In [5]:
def year_check(df_dic):
    df_keys = df_dic.keys()
    return_dic = {}
    name_list = []
    for key in df_keys:
        split_key = key.split('_')
        name = split_key[0]
        year = split_key[1]
        df_columns = df_dic[key].columns.values.tolist()
        if name not in name_list:
            name_list.append(name)
            return_dic[name] = pd.DataFrame()
        for column in df_columns:
            return_dic[name].loc[year, column] = 1
    for name in name_list:
        if return_dic[name].isnull().values.sum() != 0:
            print(pycolor.RED + name + pycolor.END)
            column_list = return_dic[name].columns.values.tolist()
            column_series = return_dic[name].isnull().any()
            for column in column_list:
                if column_series[column] == True:
                    print(column)
    return return_dic

#  テーブル間で重複するカラムの出力
デフォルトで重複は15年について調べる

In [6]:
def same_column(df_dic, conf_year = '15'):
    df_keys = df_dic.keys() 
    column_dic = {}
    name_list = []
    duplicate_list = []
    return_df = pd.DataFrame()
    for key in df_keys:
        name = key.split('_')[0]
        year = key.split('_')[1]
        if year == conf_year:
            column_dic[name] = df_dic[key].columns.values.tolist()
            name_list.append(name)
    for name in name_list:
        duplicate_list.extend(column_dic[name])
    duplicate_list = [x for x in set(duplicate_list) if duplicate_list.count(x) > 1]
    for name in name_list:
        for column in column_dic[name]:
            if column in duplicate_list:
                return_df.loc[name, column] = 1
    return return_df

# 重複したレコードの抽出
引数：DF辞書, DFのkey, 重複判定column(リスト)   
重複判定columnを先頭にしたDFを出力  

In [7]:
def duplicate(df_dic, dic_key, df_key):
    df = df_dic[dic_key]
    df = df[df.duplicated(subset = df_key, keep = False)]
    columns = list(df.columns.values)
    for column in df_key:
        columns.remove(column)
    return df.loc[:, df_key + columns]

In [8]:
cds_column = year_check(cds)

oa
CHOWUSED
LATCHDES
LATCHUSE
POSPRES
POSUSE
POSGUIDE
ve
TOWRES


In [9]:
cds_column['oa'][['CHOWUSED', 'LATCHDES', 'LATCHUSE', 'POSPRES', 'POSUSE', 'POSGUIDE']]
cds_column['ve'][['TOWRES']]

,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE
10,nan,nan,nan,nan,nan,nan
11,1,nan,nan,nan,nan,nan
12,1,1,1,1,1,1
13,1,1,1,1,1,1
14,1,1,1,1,1,1
15,1,1,1,1,1,1


,TOWRES
10,1
11,nan
12,nan
13,nan
14,nan
15,nan


In [10]:
cds['accident_11'].head()

,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,CASEID,CASENO,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSU,PSUSTRAT,RATWGT,STRATIF,TIME,VEHFORMS,YEAR,VERSION,ADMINSS
0,2.15e-314,2.2e-314,2.2e-314,2.2e-314,2.16e-314,2.17e-314,2,2.16e-314,b' 1H',1,2,2,2,2.16e-314,1,2,3,323,b'H',2.04e+03,1,2.01e+03,24,0
1,nan,nan,nan,nan,nan,nan,2,nan,b' 2D',2,3,2,2,2.16e-314,1,2,3,76.8,b'D',300,1,2.01e+03,24,0
2,nan,nan,nan,nan,nan,nan,2,nan,b' 3F',3,6,2,1,2.16e-314,1,2,3,259,b'F',1.22e+03,1,2.01e+03,24,0
3,nan,nan,nan,nan,nan,nan,2,nan,b' 4H',4,5,2,1,2.16e-314,1,2,3,408,b'H',745,1,2.01e+03,24,0
4,1,1,2.16e-314,2.2e-314,1,1,2,3,b' 5K',5,5,2,3,6,1,2,3,42.1,b'K',1.04e+03,2,2.01e+03,24,0


In [11]:
cds['event_11'].head()

,ACCSEQ,CASEID,CASENO,CLASS2,CLASS1,GADEV1,GADEV2,OBJCONT,PSU,RATWGT,STRATIF,VEHNUM,VERSION
0,1,b' 1H',1,2.16e-314,3,b'F',b'0',48,2,323,b'H',1,24
1,2,b' 1H',1,2.16e-314,3,b'L',b'0',48,2,323,b'H',1,24
2,1,b' 2D',2,2.2e-314,3,b'L',b'0',52,2,76.8,b'D',1,24
3,2,b' 2D',2,2.16e-314,3,b'R',b'N',31,2,76.8,b'D',1,24
4,1,b' 3F',3,2.16e-314,1,b'F',b'0',42,2,259,b'F',1,24


In [12]:
cds['gv_11'].head()

,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CASEID,CASENO,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,RATWGT,PREEVENT,PREMOVE,PREILOC,PREISTAB,PSU,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,STRATIF,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHNO,VEHUSE,VIN,VINLNGTH,VERSION,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT
0,1,2,nan,998,998,18,10,4,0,b' 1H',1,0,148,0,0,nan,0,1,1.24e+04,9,0,nan,nan,nan,nan,nan,0,nan,3,2,2,41,nan,50,2.00e+03,1,1,323,13,14,4,3,2,3,1,2,2,nan,0,0,0,0,0,1,8,888,0,0,0,48,b'H',0,1,6,1,2,777,0,0,1,0,b'1YVHP80C055M',17,24,0,0,0,2.00e+03,nan,3.1e+03,b'MAZD',b'M6I',NaN,b'4D',1,nan,nan,2,b'Y',b'N',b'K',b'F',b'G',nan,b'P',nan,b'U',998,98,0,0,0,0,0
1,1,2,nan,998,998,18,15,4,0,b' 2D',2,1,150,0,0,nan,0,1,1.11e+04,1,4,15,0,15,286,nan,0,998,6,2,2,42,nan,42,2e+03,2,2,76.8,13,1,4,3,2,1,1,1,2,nan,0,1,4,6,52,7,2,9,1,0,0,89,b'D',0,1,0,1,0,777,2,2,1,0,b'WDBHA29G3WA5',17,24,0,nan,nan,2e+03,nan,3.16e+03,b'MERZ',b'C28',NaN,b'4D',1,3,nan,2,b'N',b'N',b'S',b'F',b'G',nan,b'P',nan,b'N',150,4,nan,nan,nan,nan,nan
2,1,2,nan,998,998,14,22,4,0,b' 3F',3,1,117,0,1,nan,0,1,1.25e+04,1,1,6.79e-313,-22,22,246,nan,0,998,6,2,1,48,nan,38,2e+03,2,2,259,6,14,4,3,2,2,3,4,2,nan,0,0,0,0,0,1,8,888,0,0,0,56,b'F',0,1,5,1,2,777,0,0,1,0,b'JF1GC6359SH5',17,24,0,nan,nan,2e+03,nan,2.70e+03,b'SUBA',b'IL4',NaN,b'4D',1,nan,nan,5,b'N',b'Y',b'E',b'F',b'G',nan,b'P',nan,b'N',117,4,nan,nan,nan,nan,nan
3,nan,2,nan,998,998,18,nan,31,nan,b' 4H',4,2,176,nan,0,nan,0,1,1.26e+04,0,0,nan,nan,nan,nan,nan,0,nan,0,2,1,49,nan,481,1.99e+03,1,1,408,5,14,4,nan,2,2,5,4,2,nan,0,0,0,0,0,1,8,888,0,0,0,72,b'H',0,1,6,1,2,777,0,0,1,0,b'JT4VD20A2R00',17,24,0,nan,nan,1.99e+03,nan,nan,b'TOYT',b'T10',b'TLX',b'PK',1,nan,nan,6,NaN,NaN,b'M',NaN,b'G',1,b'T',44,b'N',nan,nan,nan,nan,nan,nan,nan
4,1,65,nan,280,90,18,nan,14,0,b' 5K',5,0,205,7,0,1,0,1,1.25e+04,7,0,nan,nan,nan,nan,2,0,nan,3,4,1,20,7,403,2e+03,1,1,42.1,62,14,2,1,2,2,1,2,2,1,0,0,0,0,0,1,8,888,0,0,0,72,b'K',0,1,0,1,0,777,0,0,1,0,b'1GNDT13S7423',17,24,0,0,0,2e+03,nan,nan,b'CHEV',b'TRL',b'LST',b'4W',1,3,nan,2,NaN,NaN,b'E',NaN,b'G',1,b'T',44,b'S',163,6,0,0,0,0,6


In [13]:
cds['oa_11'].head()

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,VERSION,STORIENT,GLASGOW,BLOOD,BICARB
0,17,1,7,7,97,97,1,8,2,b' 1H',1,0,0,0,7,0,0,0,2.72e-312,0,0,0,997,0,0,0,0,7,7,3,nan,0,0,2.56e-312,0,0,nan,2.82e-312,4,1,4,0,0,7,nan,1,1,7,4,8,1,nan,nan,2,323,1,1,11,1,1,97,23,b'H',5,0,1,nan,nan,nan,nan,0,0,0,0,0,0,24,1,0,1,0
1,42,nan,7,nan,nan,nan,nan,nan,nan,b' 2D',2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,1,nan,nan,4,8,nan,nan,nan,2,76.8,1,nan,nan,nan,2,nan,nan,b'D',nan,nan,1,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,24,nan,nan,nan,nan
2,50,nan,7,nan,nan,nan,nan,nan,nan,b' 2D',2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,2,nan,nan,4,8,nan,nan,nan,2,76.8,2,nan,nan,nan,1,nan,nan,b'D',nan,nan,1,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,24,nan,nan,nan,nan
3,49,nan,2,nan,nan,nan,nan,nan,nan,b' 3F',3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,1,nan,nan,4,8,nan,nan,nan,2,259,1,nan,nan,nan,1,nan,nan,b'F',nan,nan,1,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,24,nan,nan,nan,nan
4,43,nan,2,nan,nan,nan,nan,nan,nan,b' 3F',3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,2,nan,nan,4,8,nan,nan,nan,2,259,2,nan,nan,nan,2,nan,nan,b'F',nan,nan,1,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,24,nan,nan,nan,nan


In [14]:
cds['oi_11'].head()

,AIS,ASPECT90,CASEID,CASENO,DIRINJ,INJLEVEL,INJNO,INJSOU,INTRUNO,OCCNO,PSU,RATWGT,REGION90,SOUCON,SOUDAT,STRATIF,STRUSPEC,STRUTYPE,VEHNO,VERSION,BODYREG,LESION,SYSORG,REGION08,STRTYP08,STRSPC08,INJLVL08,AIS08
0,1,4,b' 5K',5,1,2,1,3,0,1,2,42.1,2,2,3,b'K',4,9,2,24,b'F',b'C',b'I',2,1,4,2,1
1,1,1,b' 6E',6,1,2,1,24,0,1,2,243,8,2,3,b'E',4,9,1,24,b'Y',b'C',b'I',8,1,4,2,1
2,1,4,b' 20F',20,1,2,1,152,0,1,2,64.5,4,1,3,b'F',4,9,2,24,b'C',b'C',b'I',4,1,4,2,1
3,1,1,b' 20F',20,1,2,2,24,0,1,2,64.5,8,2,3,b'F',4,9,2,24,b'Y',b'C',b'I',8,1,4,2,1
4,1,nan,b' 20F',20,7,2,3,697,0,1,2,64.5,1,nan,3,b'F',4,9,2,24,b'H',b'C',b'I',1,1,4,2,1


In [15]:
cds['ve_11'].head()

,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,CASEID,CASENO,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,PSU,RATWGT,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,STRATIF,SVL1,SVL2,TDD1,TDD2,VEHNO,WHEELBAS,VERSION
0,1,2,0,154,b' 1H',1,85,0,6,6,6,5,5,6,16,158,9,9,158,1,1,nan,nan,nan,0,3.4e-313,0,1,nan,b'F',b'L',nan,nan,nan,nan,1,nan,1,nan,nan,48,48,280,260,2,323,2,2,2,1,2,1,-117,131,b'Z',b'Z',b'H',b'E',b'E',b'W',b'W',1,267,24
1,1,2,0,148,b' 2D',2,35,0,0,17,33,11,2,1,146,130,9,0,998,3,2,nan,nan,nan,0,6.79e-313,0,1,nan,b'L',b'R',nan,nan,nan,nan,1,nan,1,nan,nan,52,31,270,998,2,76.8,nan,nan,nan,nan,nan,nan,nan,nan,b'F',b'D',b'D',b'E',b'A',b'N',b'O',1,269,24
2,1,nan,0,146,b' 3F',3,28,0,0,2,14,30,5,0,6,136,12,nan,136,2,nan,nan,nan,nan,0,2.72e-312,0,1,nan,b'F',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,42,nan,0,nan,2,259,nan,nan,nan,nan,nan,nan,nan,nan,b'C',NaN,b'F',b'E',NaN,b'N',NaN,1,252,24
3,1,nan,0,158,b' 5K',5,250,0,0,0,8,11,8,3,-19,287,11,nan,998,1,nan,nan,nan,nan,0,4.75e-312,0,1,nan,b'L',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,2,nan,340,nan,2,42.1,nan,nan,nan,nan,nan,nan,nan,nan,b'D',NaN,b'K',b'E',NaN,b'W',NaN,1,287,24
4,1,3,0,152,b' 5K',5,25,1,41,27,23,15,6,2,-57,144,12,0,144,5,3,nan,nan,nan,0,2.72e-312,0,1,nan,b'F',b'T',nan,nan,nan,nan,1,nan,1,nan,nan,1,31,350,998,2,42.1,nan,nan,nan,nan,nan,nan,nan,nan,b'L',b'Y',b'K',b'E',b'D',b'E',b'O',2,270,24


In [16]:
cds['vi_11'].head()

,CASEID,CASENO,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,PSU,STRATIF,VEHNO,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RATWGT,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT,VERSION
0,b' 1H',1,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,2,0,2,2,0,1,4,4,4,0,4,4,0,1,1,1,1,1,0,nan,2,b'H',1,1.36e-312,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,140,323,0,0,0,2,3.13e-312,24
1,b' 5K',5,0,0,0,0,0,1,1,1,0,1,1,1,2,1,1,1,0,1,1,1,1,2,2,2,0,2,2,2,1,3,2,3,0,2,3,3,1,1,3,1,1,1,nan,2,b'K',1,2.72e-312,2,3,21,36,2,3,11,38,1,3,21,18,1,3,21,7,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,119,42.1,0,0,0,2,3.13e-312,24
2,b' 5K',5,0,0,0,0,0,1,6,6,0,1,1,1,2,1,1,1,0,1,1,1,1,1,nan,2,0,2,2,2,1,11,nan,11,0,11,11,3,1,1,1,1,1,1,6,2,b'K',2,2.72e-312,2,1,11,15,2,1,11,16,1,1,12,16,1,1,12,15,1,1,13,16,1,1,11,6,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,129,42.1,0,0,0,1,3.48e-312,24
3,b' 6E',6,0,0,0,0,0,1,1,1,1,1,1,0,2,1,1,1,1,1,1,0,1,1,2,2,1,2,2,0,1,2,2,2,2,2,2,0,1,1,1,1,1,1,nan,2,b'E',1,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,57,243,0,0,0,1,1.65e-312,24
4,b' 7H',7,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,2,2,0,2,2,2,1,3,2,3,0,2,3,3,1,1,1,1,1,1,nan,2,b'H',1,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,161,1.13e+03,nan,nan,0,nan,1.65e-312,24


## テーブル，年毎のレコード数を表示

In [17]:
for year in range(10, 16):
    print(year)
    for file in file_name:
        print('{}:'.format(file) + str(len(cds['{}_{}'.format(file, year)])))

10
accident:4856
event:9065
gv:8921
oa:10215
oi:14276
ve:6618
vi:3475
11
accident:4278
event:8170
gv:7898
oa:8940
oi:11790
ve:5733
vi:2958
12
accident:3581
event:6755
gv:6665
oa:7402
oi:9063
ve:3509
vi:2003
13
accident:3385
event:6419
gv:6348
oa:7084
oi:8764
ve:3780
vi:2010
14
accident:2896
event:5522
gv:5445
oa:5955
oi:6975
ve:3857
vi:1901
15
accident:2633
event:4834
gv:4814
oa:5479
oi:5946
ve:3273
vi:1592


## 年毎に同一のカラムを表示

In [18]:
for year in range(10, 16):
    print(year)
    same_column(cds, str(year)).head(7)

10


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


11


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


12


...

,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


14


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


15


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


## テーブル毎の結合keyを宣言

In [19]:
merge_key = {}
for file in file_name:
    if file == 'accident' or file == 'event':
        merge_key[file] =  ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION']
    if file == 'gv' or file == 've' or file == 'vi':
        merge_key[file] = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO']
    if file == 'oa' or file == 'oi':
        merge_key[file] = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO', 'OCCNO']

## 同一の結合keyに対して重複しているレコードを表示

In [20]:
dup_dic = {}
for file in file_name:
    print(file)
    dup_dic[file] = duplicate(cds, '{}_15'.format(file), merge_key[file])
    dup_dic[file].head()

accident


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS


event


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,ACCSEQ,CLASS2,CLASS1,GADEV1,GADEV2,OBJCONT,VEHNUM
0,b' 1K',1,2,36.6,b'K',28,1,60,31,b'F',b'F',2,1
1,b' 1K',1,2,36.6,b'K',28,2,0,31,b'R',b'N',31,1
5,b' 5H',5,2,2.4e+03,b'H',28,1,20,20,b'B',b'F',3,2
6,b' 5H',5,2,2.4e+03,b'H',28,2,20,2,b'B',b'F',2,1
9,b' 8H',8,2,3.67e+03,b'H',28,1,0,3,b'R',b'0',42,1


gv


...

,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO,AIS,ASPECT90,DIRINJ,INJLEVEL,INJNO,INJSOU,INTRUNO,REGION90,SOUCON,SOUDAT,STRUSPEC,STRUTYPE,BODYREG,LESION,SYSORG,REGION08,STRTYP08,STRSPC08,INJLVL08,AIS08
0,b' 3E',3,2,375,b'E',28,1,1,1,4,1,2,1,6,0,2,2,3,2,9,b'F',b'A',b'I',2,1,2,2,1
1,b' 3E',3,2,375,b'E',28,1,1,1,7,1,2,2,6,0,2,2,3,2,9,b'F',b'A',b'I',2,1,2,2,1
2,b' 3E',3,2,375,b'E',28,1,1,1,6,1,78,3,155,0,6,2,3,2,4,b'N',b'T',b'M',6,4,2,78,1
3,b' 3E',3,2,375,b'E',28,1,1,1,8,1,2,4,6,0,2,2,3,6,9,b'F',b'L',b'I',2,1,6,2,1
4,b' 3E',3,2,375,b'E',28,1,1,1,4,1,90,5,6,0,2,2,3,10,5,b'F',b'F',b'S',2,5,10,4,1


ve


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS


vi


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT


## テーブルの結合
oi,eventテーブルを除き,2010~2015年のデータをそれぞれ結合する

In [21]:
cds_merge = {}
for year in range(10, 16):
    year = str(year)
    cds_merge[year] = cds['oa_{}'.format(year)]
    for file in [x for x in file_name if not (x == 'oa' or x == 'oi' or x == 'event')]:
        #print(file)
        cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'left')
    print(year)
    print('oa_length:' + str(len(cds['oa_{}'.format(year)])))
    print('merge_length:' + str(len(cds_merge[year])))
    cds_merge[year].head(10)

10
oa_length:10215
merge_length:10215


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,TOWRES,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,23,1,7,1,1,1,1,8,2,b' 1C',1,0,0,0,2,0,0,0,0,0,0,0,-15,0,0,0,0,1,2,3,157,0,1,2,3,4,nan,2,4,1,4,0,2,2,4,1,1,7,4,8,1,nan,nan,2,41,1,1,11,1,2,1,23,b'C',3,4,1,66,nan,1,nan,0,0,4,2,0,23,1,15,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,7,18,997,997,14,nan,4,0,0,125,7,0,1,1,1,1.26e+04,5,0,nan,nan,nan,nan,nan,0,nan,3,2,2,55,9,35,2.01e+03,2,2,6,14,4,2,2,3,1,2,1,0,1,4,2,44,3,1,20,2,0,0,56,0,1,0,1,0,777,0,4,0,b'KMHDU46D37U1',17,0,2,2,2.01e+03,nan,2.72e+03,b'HYUN',b'GST',NaN,b'4D',1,3,nan,2,b'Y',b'N',b'7',b'F',b'G',nan,b'P',nan,b'N',998,98,0,0,1,1,4,2,1,0,154,120,1,nan,nan,nan,nan,nan,nan,nan,nan,0,12,998,3,1,nan,nan,nan,0,1.36e-312,0,1,nan,b'T',b'F',nan,nan,nan,nan,1,nan,1,nan,nan,31,44,998,350,8,8,3,0,0,0,-55,158,b'Y',b'L',b'D',b'E',b'O',b'W',1,265,0,0,0,0,0,6,1,6,0,1,1,0,2,1,1,1,0,1,1,0,1,1,2,2,0,3,2,0,1,2,2,2,0,2,2,0,1,1,1,1,3,0,11,0,2,1,12,13,2,1,11,15,2,1,12,15,2,1,11,13,2,1,13,13,2,1,21,13,1,1,13,15,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,80,0,0,0,1,4.17e-312
1,31,1,7,1,1,1,1,8,4,b' 1C',1,0,0,0,2,0,2.72e-312,0,0,0,0,0,-15,0,0,0,0,1,2,3,168,0,0,2.82e-312,0,0,nan,0,4,0,0,0,0,1,4,2,1,7,4,8,3,nan,nan,2,41,2,1,13,1,1,1,23,b'C',3,0,1,82,nan,nan,nan,0,0,0,0,0,23,1,0,1,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,7,18,997,997,14,nan,4,0,0,125,7,0,1,1,1,1.26e+04,5,0,nan,nan,nan,nan

11
oa_length:8940
merge_length:8940


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,17,1,7,7,97,97,1,8,2,b' 1H',1,0,0,0,7,0,0,0,2.72e-312,0,0,0,997,0,0,0,0,7,7,3,nan,0,0,2.56e-312,0,0,nan,2.82e-312,4,1,4,0,0,7,nan,1,1,7,4,8,1,nan,nan,2,323,1,1,11,1,1,97,23,b'H',5,0,1,nan,nan,nan,nan,0,0,0,0,0,0,24,1,0,1,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,nan,nan,nan,nan,nan,nan,nan,1,2,0,154,85,0,6,6,6,5,5,6,16,158,9,9,158,1,1,nan,nan,nan,0,3.4e-313,0,1,nan,b'F',b'L',nan,nan,nan,nan,1,nan,1,nan,nan,48,48,280,260,2,2,2,1,2,1,-117,131,b'Z',b'Z',b'E',b'E',b'W',b'W',267,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,42,nan,7,nan,nan,nan,nan,nan,nan,b' 2D',2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,n

12
oa_length:7402
merge_length:7402


...

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,17,1,7,1,1,1,1,1,2,b' 1H',1,0,0,0,1,0,0,1.36e-312,0,0,0,0,-41,0,0,0,0,1,2,3,175,0,0,0,2.65e-312,0,1,0,4,1,4,0,0,1,4,1,1,7,4,8,1,0,1,2,283,1,1,11,1,1,1,23,b'H',6,0,1,73,97,1,2,0,0,0,0,0,0,0,0,0,3,0,26,1,0,1,0,0,0,0,0,0,0,2,0,6,2,1,0,1,3,825,1,2.01e+03,0,1,2,nan,998,998,19,40,20,0,1,189,0,1,5,0,1,1.25e+04,1,1,0,-41,41,1.2e+03,2,0,998,3,2,1,35,9,443,2.01e+03,1,1,5,14,4,2,3,5,4,2,1,0,0,0,0,0,1,8,888,0,0,0,89,0,1,0,1,0,nan,0,0,0,b'5N1BV28U57N1',17,0,0,0,2.01e+03,nan,nan,b'NISS',b'QST',b'Q35',b'SV',1,3,nan,2,NaN,NaN,b'S',NaN,b'G',1,b'T',42,b'U',998,98,0,0,0,0,0,1,nan,0,170,61,0,0,28,43,46,11,0,-7,171,12,nan,171,3,nan,nan,nan,nan,0,0,0,1,nan,b'F',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,42,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,b'Y',NaN,b'E',NaN,b'W',NaN,315,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,1,0,2,1,0,1,3,2,3,0,2,3,0,1,1,1,1,1,1,nan,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,138,0,0,0,5,0
1,72,nan,7,nan,nan,nan,nan,nan,nan,b' 2F',2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,1,nan,nan,4,8,nan,nan,nan,2,238,1,nan,11,nan,2,nan,nan,b'F',nan,nan,1,nan,nan,nan,nan,2.72e-312,nan,nan,nan,nan,nan,

14
oa_length:5955
merge_length:5955


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,50,nan,7,nan,nan,nan,nan,nan,nan,b' 1H',1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,1,nan,nan,4,8,nan,nan,nan,2,3.26e+03,1,nan,11,nan,2,nan,nan,b'H',nan,nan,1,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,7,nan,998,998,18,14,6,0,0,163,0,0,1,0,1,1.26e+04,1,4,-5,-13,14,146,nan,0,998,6,2,1,51,7,43,2e+03,2,2,62,1,4,4,1,3,1,2,nan,0,0,0,0,0,1,8,888,0,0,0,72,0,1,5,1,2,777,0,0,0,b'YV1SZ59H1411',17,0,nan,nan,2e+03,nan,3.83e+03,b'VOLV',b'XC0',NaN,b'SW',1,3,nan,8,b'N',b'Y',b'L',b'T',b'G',nan,b'P',nan,b'S',163,6,0,0,2,2,nan,1,nan,0,158,118,0,3,1,4,3,1,3,15,160,1,nan,160,1,nan,nan,nan,nan,0,0,0,1,nan,b'F',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,48,nan,20,nan,nan,nan,nan,nan,nan,nan,nan,nan,b'D',NaN,b'M',NaN,b'W',NaN,276,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,14,nan,7,nan,nan,nan,nan,nan,nan,b' 1H',1,na

15
oa_length:5479
merge_length:5479


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,26,nan,7,nan,nan,nan,nan,nan,nan,b' 1K',1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,1,nan,nan,4,8,nan,nan,nan,2,36.6,1,nan,11,nan,1,nan,nan,b'K',nan,nan,1,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,28,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,nan,7,2,2,2,1,3,1.51e+03,2,2.02e+03,0,1,50,nan,280,90,14,50,31,0,0,231,4,0,nan,0,1,1.24e+04,4,0,nan,nan,nan,nan,nan,0,nan,6,2,1,7,7,482,2e+03,1,1,5,14,2,3,3,3,4,2,nan,0,1,1,7,2,6,2,10,1,0,0,89,0,1,0,1,0,777,0,2,0,b'3B7KF23Z5VM5',17,0,nan,nan,2e+03,nan,nan,b'DODG',b'RPC',b'250',b'PC',1,nan,nan,4,NaN,NaN,b'M',NaN,b'G',2,b'T',44,b'N',nan,63,nan,nan,nan,nan,nan,1,2,0,170,138,0,70,68,62,51,40,33,0,190,12,0,190,3,2,nan,nan,nan,0,0,0,1,nan,b'F',b'R',nan,nan,nan,nan,1,nan,1,nan,nan,2,31,350,998,nan,nan,nan,nan,nan,nan,nan,nan,b'D',b'Y',b'E',b'A',b'W',b'O',353,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,33,nan,7,nan,nan,nan,nan,nan,nan,b' 2D',2,nan,

## 特徴量のドロップ
結合用の特徴量・AISを元にした特徴量・年によってデータが存在しない特徴量をドロップ

In [22]:
cds_prepro = {}
drop_columns = ['CASEID', 'CASENO', 'VEHNO', 'OCCNO', 'RATWGT', 'STRATIF', 'VERSION', 'AAIS',\
                'AAIS08', 'VAIS', 'VAIS08', 'MAIS', 'MAIS08', 'INJNUM08', 'ISS08', 'TOWRES', \
                'VIN', 'AINJSER', 'AINJSER8', 'AINJURED', 'AINJURD8', 'VINJSER', 'VINJSER8', \
                'VINJURED', 'VINJURD8', 'INJNUM', 'LATCHDES', 'LATCHUSE', 'POSPRES', 'POSUSE', \
                'POSGUIDE','CHOWUSED', 'FUELTYP1', 'FUELTYP2']#INJSEVは警察の報告でありAISデータを使用していない
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_merge[year]
    year_columns = cds_prepro[year].columns.values.tolist()
    for column in drop_columns:
        if column in year_columns:
            cds_prepro[year] = cds_prepro[year].drop(columns = [column])

## 値の変更
同一のものを指しているにも関わらず年によって値が違うものを修正

In [23]:
for year in range(10, 16):
    year = str(year)
    cds_prepro[year]['CLIMATE'] = cds_prepro[year]['CLIMATE'].replace('13', '20') 

## ISSデータが存在しないレコードのドロップ
推定対象のISSが存在しないレコードのドロップ

In [24]:
#ISSのnanをdropし，年毎のレコード数をprint
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_prepro[year].dropna(subset = ['ISS'])
    print(str(year) + ':' + str(len(cds_prepro[year])))

10:6229
11:5225
12:4094
13:3819
14:3253
15:2901


## 論文と年以外の条件を同じに
Vehicle Modelの0~490が相当

In [56]:
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_prepro[year].query('0 <= MODEL <= 490 & AGE >= 15 & 2001 <= MODELYR <= 2015 & EJECTION == 0')
    print(str(year) + ':' + str(len(cds_prepro[year])))

10:5362
11:918
12:2549
13:3345
14:2891
15:2525


In [106]:
col = cds_prepro['15'].query('MODEL > 500')
path = os.path.dirname(os.path.abspath('__file__'))
col.to_csv(os.path.join(path, 'column.csv'))

## 15年の要約統計量

In [103]:
cds_prepro['15'].describe()

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJSEV,ISS,BAGMAINT,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,SEATRACK,TREATMNT,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,INTGREST,STORIENT,GLASGOW,BLOOD,BICARB,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VINLNGTH,DRUGS,VINMODYR,MCYCLDS,VEHWGT,ROOF1,ROOF2,ROOF3,ANTILOCK,WGTCDTR,WHLDRWHL,otvehwgt,otbdytyp,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
count,2525,2179,2475,2179,2179,2176,1833,2179,2017,2501,2525,2525,2179,2525,2525,2525,2525,2525,2525,2500,2179,2525,2525,2525,2258,2179,2179,1927,1865,2395,2492,2525,1338,2282,1778,2140,2525,2463,2179,1712,2193,2193,2406,2525,2182,1402,1379,2525,2525,1799,2523,1954,2524,2177,1782,1743,2476,1887,1597,1808,1703,2525,2525,1904,1946,2364,2236,2364,2131,2481,2525,1853,2525,2454,2525,2525,2511,2525,2525,2525,2156,2524,380,2516,2514,2522,1542,2525,1906,2521,2466,2030,2525,1397,2445,2525,2510,2525,2525,1409,1409,1409,1409,1734,2306,1409,2525,2525,2519,2525,1737,2525,2525,2525,2525,2521,2522,2520,2250,2525,2522,2487,2525,1836,2525,2495,2495,2495,2495,2493,2494,2435,2495,2424,2343,2502,2523,2525,2525,2525,2524,2074,2525,2495,2516,2449,2453,2525,77,1623,2495,1497,128,2495,976,976,2024,2152,2475,2031,759,2030,2021,1520,2031,1354,1354,1333,1333,1333,1333,1354,1354,1924,693,1824,1690,550,16,0,14,1986,2031,2028,1887,42,8,0,8,0,1886,42,0,2031,756,1925,693,288,288,287,287,276,276,288,288,2029,2031,2031,2031,2031,2031,1946,1918,1937,2017,1927,1940,1996,1938,1875,1833,1870,2016,1850,1867,1988,1814,2012,1905,1941,2021,1913,1938,1986,2022,1853,1837,1860,2018,1836,1853,1984,1949,1859,1888,1842,1867,1969,490,1942,1704,493,493,493,493,373,373,373,373,293,293,293,293,211,211,211,211,162,162,162,162,137,137,137,137,110,110,110,110,80,80,80,80,65,65,65,65,51,51,51,51,1285,1776,1776,1626,1583,2031
mean,39.15,0.9215,3.173,3.661,50.25,53.78,0.9651,3.593,2.761,0.2231,0.1101,0.1263,3.968,5.459e-314,1.715e-312,0,0,0,0,0.168,613.9,0,0,0,0.02879,3.845,4.218,2.919,170.7,0.7503,1.248,3.083,0.8677,4.011,0.8048,3.361,0,0.8965,4.275,3.464,0.8003,3.939,3.882,7.945,1.338,0.1583,1.071,41.52,1.247,1.418,12.15,1.

In [42]:
cds_prepro['15'].head(50)

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJSEV,ISS,BAGMAINT,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,SEATRACK,TREATMNT,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,INTGREST,STORIENT,GLASGOW,BLOOD,BICARB,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VINLNGTH,DRUGS,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
2,57,1,7,7,97,97,1,1,2,0,0,0,7,0,0,0,0,0,0,0,997,0,0,0,0,7,7,3,152,0,2,3,1,4,1,4,0,2,7,2,1,7,4,8,1,0,1,2,1,5,11,1,2.00038,97,22,1,4,70,7,1,2,0,0,0,1,15,1,1,2,4,3,2,1,1,1,3,1355,2,2015,0,1,22,nan,350,350,18,25,4,0,0,154,0,0,1,0,1,12401,3,4,0,21,21,404,2,0,998,3,4,1,63,1,34,2014,1,1,53,5,1,1,2,1,2,2,1,3,0,0,0,0,1,8,888,0,0,0,72,0,1,0,1,0,777,0,1.043e-312,0,17,0,2014,nan,3385,b'KIA',b'OSX',NaN,b'4D',1,3,nan,2,b'Y',b'N',b'7',b'F',b'G',nan,b'P',nan,b'S',143,4,4,1,nan,0,160,144,0,18,21,26,32,23,17,0,144,6,nan,144,4,nan,nan,nan,nan,0,0,0,1,nan,b'B',NaN,nan,nan,nan,nan,1,nan,nan,2,nan,180,nan,nan,nan,nan,nan,nan,nan,nan,nan,b'D',NaN,b'E',NaN,b'W',NaN,280,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,2,0,2,2,0,1,2,2,2,0,2,2,0,1,1,1,1,1,0,nan,0,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,12.0004,0,0,5,1,0
5,34,1,7,7,97,97,1,1,4,0,0,0,7,0,0,0,0,0,0,0,997,0,0,0,0,7,7,3,183,0,1,6,1,4,1,4,0,1,7,2,0,0,4,8,1,0,2,2,1,1,11,1,1.00006,97,23,6,4,91,21,7,2,0,0,0,1,15,1,21,2,4,1,2,1,6,1,3,1244,2,2015,0,1,65,nan,160,330,18,22,4,0,0,105,0,0,1,0,1,12725,1,4,13,-15,20,234,2,0,998,3,2,1,49,9,51,2007,2,2,62,14,1,1,2,2,1,2,7,0,0,0,0,0,1,8,888,0,0,0,72,0,1,0,1,0,777,0,3.47668e-313,0,17,0,2007,nan,2288,b'TOYT',b'YAR',NaN,b'4D',1,nan,nan,5,b'Y',b'N',b'E',b'F',b'G',nan,b'P',nan,b'O',152,4,4,1,nan,0,148,220,0,2,3,3,6,15,0,80,255,11,nan,998,3,nan,nan,nan,nan,0,0,0,1,nan,b'L',NaN,nan,nan,nan,nan,1,nan,nan,1,nan,320,nan,nan,nan,nan,nan,nan,nan,nan,nan,b'Y',NaN,b'A',

# exp

In [27]:
exp_column = year_check(exp)

airbag
RECTYPE
bagseat
RECTYPE
childseat
STRATIF
VERSION
RECTYPE
seatloc
VERSION
RECTYPE
POSGUIDE


In [28]:
pd.DataFrame(exp_column['airbag']['RECTYPE'])
pd.DataFrame(exp_column['bagseat']['RECTYPE'])
exp_column['childseat'][['STRATIF','VERSION']]
exp_column['seatloc'][['VERSION' ,'RECTYPE' ,'POSGUIDE']]

,RECTYPE
11,nan
12,nan
13,1
14,1
15,1


,RECTYPE
11,nan
12,nan
13,1
14,1
15,1


,STRATIF,VERSION
11,nan,nan
12,nan,nan
13,1,1
14,1,1
15,1,1


,VERSION,RECTYPE,POSGUIDE
11,nan,nan,nan
12,1,1,1
13,1,1,1
14,1,1,1
15,1,1,1


In [29]:
exp['airbag_11']

,SCASEID,PSU,CASENO,VEHNO,BAGNO,STRATIF,RATWGT,VERSION,BAGLOC,BAGFUNC,BAGTYPE,BAGDEPLY,BAGFAIL,SWITCHTYP,SWTCHSET,BAGFLOPN,BAGFLDAM,BAGDAMAG,BAGDAMSO,PREVACC,BAGMAINT,BAGEVENT,BAGCDC,DVBAG
0,7.6e+08,2,1,1,1,b'H',323,24,1,1,4,7,0,0,7,60,60,60,60,9,9,60,60,997
1,7.6e+08,2,1,1,2,b'H',323,24,2,1,4,7,0,0,7,60,60,60,60,0,0,0,0,995
2,7.6e+08,2,1,1,3,b'H',323,24,5,1,4,7,0,0,7,60,60,60,60,9,9,60,60,997
3,7.6e+08,2,1,1,4,b'H',323,24,5,1,4,7,0,0,7,60,60,60,60,0,0,0,0,995
4,7.6e+08,2,1,1,5,b'H',323,24,7,1,4,7,0,0,7,60,60,60,60,9,9,60,60,997
5,7.6e+08,2,1,1,6,b'H',323,24,7,1,4,7,0,0,7,60,60,60,60,0,0,0,0,995
6,7.6e+08,2,5,1,1,b'K',42.1,24,1,1,1,7,0,0,7,60,60,60,60,1,0,60,60,997
7,7.6e+08,2,5,1,2,b'K',42.1,24,3,1,1,7,0,0,7,60,60,60,60,0,0,0,0,995
8,7.6e+08,2,5,1,3,b'K',42.1,24,5,1,1,7,0,0,7,60,60,60,60,1,0,60,60,997
9,7.6e+08,2,5,1,4,b'K',42.1,24,5,1,1,7,0,0,7,60,60,60,60,0,0,0,0,995


In [30]:
for expand in expand_name:
    print('{}:'.format(expand) + str(len(exp['{}_15'.format(expand)])))

airbag:9104
bagseat:11863
childseat:129
seatloc:8654
tire:12856
tiredmg:13120


In [31]:
same_column(exp)

,SCASEID,PSU,CASENO,VEHNO,BAGNO,STRATIF,RATWGT,VERSION,RECTYPE,SEATPOS,TLOC
airbag,1,1,1,1,1,1,1,1,1,nan,nan
bagseat,1,1,1,1,1,1,1,1,1,nan,nan
childseat,1,1,1,1,nan,1,1,1,1,1,nan
seatloc,1,1,1,1,nan,1,1,1,1,1,nan
tire,1,1,1,1,nan,1,1,1,1,nan,1
tiredmg,1,1,1,1,nan,1,1,1,1,nan,1


In [32]:
for expand in expand_name:
    print('{}:'.format(expand) + str(len(exp['{}_15'.format(expand)])))

airbag:9104
bagseat:11863
childseat:129
seatloc:8654
tire:12856
tiredmg:13120


In [33]:
same_column(exp)

,SCASEID,PSU,CASENO,VEHNO,BAGNO,STRATIF,RATWGT,VERSION,RECTYPE,SEATPOS,TLOC
airbag,1,1,1,1,1,1,1,1,1,nan,nan
bagseat,1,1,1,1,1,1,1,1,1,nan,nan
childseat,1,1,1,1,nan,1,1,1,1,1,nan
seatloc,1,1,1,1,nan,1,1,1,1,1,nan
tire,1,1,1,1,nan,1,1,1,1,nan,1
tiredmg,1,1,1,1,nan,1,1,1,1,nan,1


#  以下挙動確認用

In [34]:
hoge = pd.DataFrame({'key1':[1, 1, 1],
                     'key2':[3,3,3],
                     'key3':[1, 3, 3],
                     'data_x':['a', 'b', 'c']})
hoge.head()
hoge2 = pd.DataFrame({'key1':[1, 1, 3],
                      'key2':[3, 3, 5],
                      'key3':[1, 3, 4],
                      'data_y':['d', 'e', 'f']})
hoge2.head()
hoge3 = pd.DataFrame({'key1':[1, 1, 2],
                      'data':['g', 'h', 'i']})
#hoge3.head()

,key1,key2,key3,data_x
0,1,3,1,a
1,1,3,3,b
2,1,3,3,c


,key1,key2,key3,data_y
0,1,3,1,d
1,1,3,3,e
2,3,5,4,f


In [35]:
hoge['data_x'] = hoge['data_x'].replace('a', 'e')

In [36]:
hoge.head()

,key1,key2,key3,data_x
0,1,3,1,e
1,1,3,3,b
2,1,3,3,c


In [37]:
#pd.merge(hoge, hoge2, on = ['key1', 'key2'], how = 'outer')
#pd.merge(hoge, hoge2, on = ['key1', 'key2'], how = 'inner')
pd.merge(hoge, hoge2, on = ['key1', 'key2', 'key3'], how = 'left')

,key1,key2,key3,data_x,data_y
0,1,3,1,e,d
1,1,3,3,b,e
2,1,3,3,c,e


In [38]:
test = pd.merge(hoge, hoge2, on = 'key1', how = 'inner')
test.head()
test = pd.merge(test, hoge3, on = 'key1', how = 'inner')
test.head(10)

,key1,key2_x,key3_x,data_x,key2_y,key3_y,data_y
0,1,3,1,e,3,1,d
1,1,3,1,e,3,3,e
2,1,3,3,b,3,1,d
3,1,3,3,b,3,3,e
4,1,3,3,c,3,1,d


,key1,key2_x,key3_x,data_x,key2_y,key3_y,data_y,data
0,1,3,1,e,3,1,d,g
1,1,3,1,e,3,1,d,h
2,1,3,1,e,3,3,e,g
3,1,3,1,e,3,3,e,h
4,1,3,3,b,3,1,d,g
5,1,3,3,b,3,1,d,h
6,1,3,3,b,3,3,e,g
7,1,3,3,b,3,3,e,h
8,1,3,3,c,3,1,d,g
9,1,3,3,c,3,1,d,h


In [39]:
hogehoge = pd.DataFrame()
pd.merge(hoge, hogehoge, on = ['key1'])

KeyError: 'key1'